In [880]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [881]:
import pandas as pd
import numpy as np

import inventario_finanzas_int.funciones_auxiliares as fa

pd.set_option("display.max_columns", None)

In [882]:

# lee mobiliarios
df_mobiliario = pd.read_csv("../data/processed/df_procesada_mobiliarios.csv")

# Lee equipos medicos y los ordena
df_equipos_medicos = pd.read_csv("../data/processed/df_procesada_equipos_medicos.csv")
df_equipos_medicos = df_equipos_medicos.sort_values(
    ["piso", "unidadservicio_clinico", "ubicacion_unidad", "bien"],
    ascending=[True, True, False, True],
)

# lee Industriales, y elimina las columnas completamente vacias
df_industriales = pd.read_csv("../data/processed/df_procesada_industriales.csv")
df_industriales = df_industriales.dropna(axis=1, how="all")
df_industriales = df_industriales.sort_values(
    ["piso", "unidadservicio_clinico", "ubicacion_unidad", "bien"],
    ascending=[True, True, False, True],
)

# Lee Informaticos
df_informaticos = pd.read_csv("../data/processed/df_procesada_informaticos.csv")

In [883]:
union_bienes_con_informatico = pd.concat(
    [df_mobiliario, df_equipos_medicos, df_industriales, df_informaticos]
)

# Deja solamente las columnas utiles
columnas_utiles = [
    "correlativo_antiguo",
    "n_inventario_2025",
    "bien",
    "marca",
    "modelo",
    "serie",
    "tipo_bien",
    "piso",
    "unidadservicio_clinico",
    "ubicacion_unidad",
    "propiedad",
    "observaciones",
]

# Agrega el conteo de bienes
union_bienes_con_informatico["conteo"] = 1

conteo_equipos = pd.pivot_table(
    union_bienes_con_informatico,
    columns="tipo_bien",
    index="unidadservicio_clinico",
    values="conteo",
    fill_value=0,
    aggfunc="sum",
)

In [884]:
unidades_a_preguntar = [
    "4 NORTE",
    "CAMAS MEDIAS RESPIRATORIO",
    "ENFERMERA COORDINADORA",
    "FARMACIAS",
    "JEFE MEDICO CARDIOCIRUGIA",
    "MEDICINA RESPIRATORIA",
    "MQ",
    "RECEPCION DE MUESTRA",
    "SALA CLINICA 3ERO SUR",
    "SALA DE MEDICOS CARDIOCIRUGIA",
    "SALA MEDICOS",
    "SALA RESIDENTE 3RO NORTE",
    "SECRETARIA CARDIOCIRUGIA",
    "SECRETARIA ENFERMERIA UTI 2",
    "UCHILE",
    "UCI 5",
    "UNIDAD PACIENTE CRITICO 3",
    "UPC",
    "UCI-UPC",
    "UCI-UTI",
    "UPC 3",
    "UPC 4",
    "UPC 5 PISO",
    "UPC-UCI",
    "UTI 4",
    "UCI 5 NORTE",
]

## Asignacion de Correlativos

Se hara de forma separada

In [887]:
GLOSAS_CON_CORRELATIVOS = [
    "INT",
    "INT (SSMO)",
    "INT (MEL)",
    "INT (MINSAL)",
    "INT (DONACION)",
    "COMODATO",
]

In [896]:
def asignar_correlativos(
    df, correlativo_inicial, glosas_con_correlativos, columna_salida="correlativo_2025"
):
    """
    Asigna correlativos secuenciales a los registros cuyo valor en la columna 'propiedad'
    esté presente en `glosas_con_correlativos`.

    Parámetros:
    - df: DataFrame con los datos.
    - correlativo_inicial: número entero desde el cual se empezarán a asignar los correlativos.
    - glosas_con_correlativos: lista de valores que indican las filas a las que se les debe asignar 
    correlativo.
    - columna_salida: nombre de la columna donde se asignarán los correlativos. Por defecto: 
    'correlativo_2025'.

    Retorna:
    - df_copy: copia del DataFrame con la columna de correlativos asignada.
    - ultimo_correlativo: último correlativo numérico asignado.
    """
    df_copy = df.copy()

    # Identifica las filas a las que se asignará correlativo
    mask = df_copy["propiedad"].isin(glosas_con_correlativos)
    df_filtrado = df_copy[mask]

    # Cantidad de registros a asignar
    cantidad = len(df_filtrado)

    # Genera correlativos
    correlativos = [f"2025-{i}" for i in range(correlativo_inicial, correlativo_inicial + cantidad)]

    # Asigna correlativos
    df_copy.loc[mask, columna_salida] = correlativos

    # Reemplaza "nan" como string por np.nan (por si acaso)
    df_copy[columna_salida] = df_copy[columna_salida].replace("nan", np.nan)

    # Calcula el último correlativo asignado
    ultimo_correlativo = (
        correlativo_inicial + cantidad - 1 if cantidad > 0 else correlativo_inicial - 1
    )

    return df_copy, ultimo_correlativo

In [912]:
# Asigna correlativos a equipos medicos
df_equipos_medicos, ultimo_correlativo_eq_med = asignar_correlativos(
    df_equipos_medicos, 4016, GLOSAS_CON_CORRELATIVOS, "n_inventario_2025"
)

# Asigna correlativos a industriales
df_industriales, ultimo_correlativo_ind = asignar_correlativos(
    df_industriales, ultimo_correlativo_eq_med + 1, GLOSAS_CON_CORRELATIVOS, "n_inventario_2025"
)

# Asigna correlativos a informaticos
df_informaticos, ultimo_correlativo_inf = asignar_correlativos(
    df_informaticos, ultimo_correlativo_ind + 1, GLOSAS_CON_CORRELATIVOS, "n_inventario_2025"
)

In [910]:
df_industriales

,correlativo_antiguo,bien,marca,modelo,serie,propiedad,piso,unidadservicio_clinico,ubicacion_unidad,observaciones,tipo_bien,correlativo_2025,n_inventario_2025
69,1499-5625,EQUIPO DE CLIMA,MIDEA,S/D,S/D,INT,-1,ANATOMIA PATOLOGICA,SECRETARIA,OPERATIVO,EQUIPO INDUSTRIAL,2025-4984,2025-4984
67,1499-5672,EQUIPO DE CLIMA,ANWO,S/D,S/D,INT,-1,ANATOMIA PATOLOGICA,SALA HISTOTECNICA,OPERATIVO,EQUIPO INDUSTRIAL,2025-4985,2025-4985
77,NaN,DISPENSADOR DE AGUA,AQUAFREE,S/D,S/D,COMODATO,-1,ANATOMIA PATOLOGICA,SALA ESTAR TECNICOS,OPERATIVO,EQUIPO INDUSTRIAL,2025-4986,2025-4986
76,NaN,HORNO ELECTRICO,OSTER,S/D,S/D,PROPIEDAD DEL FUNCIONARIO,-1,ANATOMIA PATOLOGICA,SALA ESTAR TECNICOS,OPERATIVO,EQUIPO INDUSTRIAL,NaN,NaN
75,NaN,MICROONDAS,SOMELA,S/D,S/D,PROPIEDAD DEL FUNCIONARIO,-1,ANATOMIA PATOLOGICA,SALA ESTAR TECNICOS,OPERATIVO,EQUIPO INDUSTRIAL,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
29,NaN,TANQUE HIDRONEUMATICO HORIZONTAL 50 LITROS,S/D,S/D,S/D,INT,PATIO,ESTERILIZACION,PLANTA DE AGUA,OPERATIVO,EQUIPO INDUSTRIAL,2025-5846,2025-5846
25,NaN,TANQUE HIDRONEUMATICO VERTICAL 100 LITROS,S/D,S/D,S/D,INT,PATIO,ESTERILIZACION,PLANTA DE AGUA,OPERATIVO,EQUIPO INDUSTRIAL,2025-5847,2025-5847
64,NaN,EQUIPO DE CLIMA,KENDAL,S/D,S/D,INT,PATIO,MEDICINA FISICA Y REHABILITACION,CONTAINER,OPERATIVO,EQUIPO INDUSTRIAL,2025-5848,2025-5848
65,1499-465,FRIGOBAR,S/D,S/D,S/D,INT,PATIO,MEDICINA FISICA Y REHABILITACION,CONTAINER,OPERATIVO,EQUIPO INDUSTRIAL,2025-5849,2025-5849


In [ ]:
ULTIMO_CORRELATIVO = 4015 + 1
# Identifica los registros a los que hay que asignar correlativo
mask_correlativo_medicos = df_equipos_medicos["propiedad"].isin(
    GLOSAS_CON_CORRELATIVOS
)
df_equipos_medicos_con_correlativos = df_equipos_medicos[mask_correlativo_medicos]

# Asigna correlativos a los registros que requieren correlativo
correlativos_equipos_medicos = [
    f"2025-{i}"
    for i in range(
        ULTIMO_CORRELATIVO, ULTIMO_CORRELATIVO + len(df_equipos_medicos_con_correlativos)
    )
]
df_equipos_medicos.loc[mask_correlativo_medicos, "correlativo_2025"] = correlativos_equipos_medicos
df_equipos_medicos["correlativo_2025"] = df_equipos_medicos["correlativo_2025"].replace(
    "nan", np.nan
)

In [890]:
# Aisla el ultimo correlativo asignado
ULTIMO_CORRELATIVO_MEDICOS = int(
    df_equipos_medicos["correlativo_2025"].dropna().iloc[-1].split("-")[-1]
)

PRIMER_CORRELATIVO_INDUSTRIALES = ULTIMO_CORRELATIVO_MEDICOS + 1
# Identifica los registros a los que hay que asignar correlativo
mask_correlativos_ind = df_industriales["propiedad"].isin(GLOSAS_INDUSTRIALES_CON_CORRELATIVO)
df_industriales_con_correlativos = df_industriales[mask_correlativos_ind]

# Asigna los correlativos a los registros que requieren correlativo
correlativos_industriales = [
    f"2025-{i}"
    for i in range(
        PRIMER_CORRELATIVO_INDUSTRIALES,
        PRIMER_CORRELATIVO_INDUSTRIALES + len(df_industriales_con_correlativos),
    )
]

df_industriales.loc[mask_correlativos_ind, "correlativo_2025"] = correlativos_industriales
df_industriales["correlativo_2025"] = df_industriales["correlativo_2025"].replace("nan", np.nan)

In [891]:
# Genera la matriz total de bienes
matriz_total_correlativos = pd.concat([df_equipos_medicos, df_industriales])
matriz_total_correlativos = matriz_total_correlativos.drop(columns="unidad_hoja")

# Reordena columnas de matriz total
orden_columnas = ["correlativo_antiguo", "correlativo_2025"] + list(
    matriz_total_correlativos.columns.drop(["correlativo_antiguo", "correlativo_2025"])
)
matriz_total_correlativos = matriz_total_correlativos[orden_columnas]

In [892]:
# Exporta asignando correlativos
df_equipos_medicos.to_excel("../data/interim/matriz_equipos_medicos_2025.xlsx", index=False)
df_industriales.to_excel("../data/interim/matriz_industriales_2025.xlsx", index=False)

In [893]:
fa.guardar_dataframe_como_tabla_excel(
    matriz_total_correlativos,
    "../data/interim/matriz_bienes_INT_2025.xlsx",
    nombre_tabla="MatrizBienes",
    estilo_tabla="TableStyleMedium1"
)

✅ Archivo guardado exitosamente en: ../data/interim/matriz_bienes_INT_2025.xlsx


## Exportar bases para etiquetas

In [894]:
# Aisla la matriz de equipos medicos
for unidad in matriz_total_correlativos.query("tipo_bien == 'EQUIPO MEDICO'")[
    "unidadservicio_clinico"
].unique():
    # Aisla la unidad
    df_equipos_medicos_unidad = df_equipos_medicos.query("unidadservicio_clinico == @unidad")

    fa.guardar_dataframe_como_tabla_excel(
        df_equipos_medicos_unidad,
        f"../data/interim/etiquetas_equipos_medicos/ETIQUETAS EQUIPOS MEDICOS {unidad}.xlsx",
        nombre_tabla="MatrizBienes",
        estilo_tabla="TableStyleMedium1",
    )

✅ Archivo guardado exitosamente en: ../data/interim/etiquetas_equipos_medicos/ETIQUETAS EQUIPOS MEDICOS MEDICINA FISICA Y REHABILITACION.xlsx
✅ Archivo guardado exitosamente en: ../data/interim/etiquetas_equipos_medicos/ETIQUETAS EQUIPOS MEDICOS CONSULTORIO EXTERNO.xlsx
✅ Archivo guardado exitosamente en: ../data/interim/etiquetas_equipos_medicos/ETIQUETAS EQUIPOS MEDICOS LABORATORIO.xlsx
✅ Archivo guardado exitosamente en: ../data/interim/etiquetas_equipos_medicos/ETIQUETAS EQUIPOS MEDICOS UNIDAD DE APOYO FARMACIA.xlsx
✅ Archivo guardado exitosamente en: ../data/interim/etiquetas_equipos_medicos/ETIQUETAS EQUIPOS MEDICOS FUNCION PULMONAR.xlsx
✅ Archivo guardado exitosamente en: ../data/interim/etiquetas_equipos_medicos/ETIQUETAS EQUIPOS MEDICOS UMT.xlsx
✅ Archivo guardado exitosamente en: ../data/interim/etiquetas_equipos_medicos/ETIQUETAS EQUIPOS MEDICOS MQ 3 NORTE.xlsx
✅ Archivo guardado exitosamente en: ../data/interim/etiquetas_equipos_medicos/ETIQUETAS EQUIPOS MEDICOS MQ 3 SUR.xl

In [895]:
matriz_total_correlativos.query("tipo_bien == 'EQUIPO MEDICO'")["propiedad"].value_counts()

propiedad
INT                         872
COMODATO                     91
ARRIENDO                      4
PRESTAMO HOSMET               3
INT (SSMO)                    3
INT (MINSAL)                  2
REVISAR DONACION GRIFOLS      1
PROPIEDAD DRA. LINACRE        1
Name: count, dtype: int64